In [15]:
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os


In [16]:
def plot_elevation_alongshore(data, file_path, dem_name):
    dy = data.geometry.y.max() - data.geometry.y.min()
    dx = data.geometry.x.max() - data.geometry.x.min()
    # theta = np.arctan(dy / dx) * (180 / np.pi)
    theta = 31
    # print(f'Angle of the shore: {theta} degrees')
    data["adjusted_x"] = data["x"] / np.cos(
        np.radians(theta)
    )  # Assuming 'x' is already defined in data
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x="adjusted_x", y="elevation", data=data)
    plt.xlabel("Alongshore Distance (m)")
    plt.ylabel("Elevation (m)")
    plt.title(f"Elevation vs Adjusted Alongshore Distance - {dem_name}")
    plot_file_path = file_path.replace(".geojson", f"_alongshore_{dem_name}.svg")
    plt.savefig(plot_file_path)
    plt.close()


def plot_elevation_pdf(data, file_path, dem_name):
    plt.figure(figsize=(10, 6))
    sns.histplot(data["elevation"], kde=True, stat="density")
    plt.xlabel("Elevation (m)")
    plt.ylabel("Density")
    plt.title(f"Normalized PDF of Elevations - {dem_name}")
    plot_file_path = file_path.replace(".geojson", f"_pdf_{dem_name}.svg")
    plt.savefig(plot_file_path)
    plt.close()


def plot_3d_elevation_view(data, file_path, dem_name):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(
        x="x", y="y", size="elevation", hue="elevation", palette="coolwarm", data=data
    )
    plt.xlabel("Alongshore Distance (m)")
    plt.ylabel("Cross-shore Distance (m)")
    plt.title(f"3D View of Elevation (X, Y, Elevation) - {dem_name}")
    plot_file_path = file_path.replace(".geojson", f"_3d_view_{dem_name}.svg")
    plt.savefig(plot_file_path)
    plt.close()


def plot_elevation_ecdf(data, file_path, dem_name):
    plt.figure(figsize=(10, 6))
    sorted_elevation = np.sort(data["elevation"])
    print(max(sorted_elevation))
    ecdf = np.arange(1, len(sorted_elevation) + 1) / len(sorted_elevation)
    plt.step(sorted_elevation, ecdf)
    plt.xlabel("Elevation (m)")
    plt.ylabel("ECDF")
    plt.title(f"Empirical Cumulative Distribution Function of Elevation - {dem_name}")
    plot_file_path = file_path.replace(".geojson", f"_ecdf_{dem_name}.svg")
    plt.savefig(plot_file_path)
    plt.close()


In [17]:
def process_geojson_files(geojson_dict):
    for file_path, dem_name in geojson_dict.items():
        if os.path.exists(file_path):
            geo_data = gpd.read_file(file_path)

            # Assuming 'elevation' is directly a column in geo_data
            # If 'x' and 'y' are not columns, calculate them from the geometry

            geo_data["x"] = (geo_data.geometry.x - geo_data.geometry.x.min()) * 1
            geo_data["y"] = (geo_data.geometry.y - geo_data.geometry.y.min()) * 1

            # No need to extract 'elevation' from 'properties' as previously attempted
            # Directly use the 'elevation' column

            # plot_elevation_alongshore(geo_data, file_path, dem_name)
            plot_elevation_pdf(geo_data, file_path, dem_name)
            plot_3d_elevation_view(geo_data, file_path, dem_name)
            plot_elevation_ecdf(geo_data, file_path, dem_name)
            print(f"Processed {dem_name}")


In [18]:
geojson_dict = {
    "max_elevations_DEM_2001.geojson": "DEM 2001",
    "max_elevations_DEM_2002.geojson": "DEM 2002",
    "max_elevations_DEM_2006.geojson": "DEM 2006",
    "max_elevations_DEM_2009.geojson": "DEM 2009",
    "max_elevations_DEM_2016.geojson": "DEM 2016",
    "max_elevations_DEM_2018.geojson": "DEM 2018",
    "max_elevations_DEM_2018_19_post_harvey.geojson": "DEM 2018_19 post Harvey",
}


process_geojson_files(geojson_dict)


3.0067625045776367
Processed DEM 2001
1.9588885307312012
Processed DEM 2002
2.229612112045288
Processed DEM 2006
2.2081236839294434
Processed DEM 2009
3.471081495285034
Processed DEM 2016
2.9809350967407227
Processed DEM 2018
2.989924669265747
Processed DEM 2018_19 post Harvey
